## Modelo Matemático

In [10]:
import pulp as pulp
import pandas as pd

### Conjuntos

In [11]:
# Conjuntos:
I = [1, 2, 3, 4]           # proveedores
J = list(range(1, 11))     # especies 1..10
T = range(1, 10)            # por ejemplo 5 períodos (ajusta según tu caso)
P = list(range(1, 5))     # polígonos 1..19
L = list(range(1, 4))      # viajes 1..3 (ajusta según tu caso)
P = list(range(1,5))   # polígonos 1..19
Nodos = P               # en este caso, almacen a=1 también está en P
a = 1                      # índice del almacén

### Parámetros:

In [12]:
# Parámetros
ruta_csv = "distancias4Poligonos.csv"
df_dist = pd.read_csv(ruta_csv)
indice = df_dist.index.tolist()
columnas = df_dist.columns.tolist()

# Costo unitario C_{i,j}
C = {
    (1,1): 1e6,   # proveedor 1 no vende lechuguilla → asignamos costo muy alto
    (2,1): 1e6,   
    (3,1): 1e6,
    (4,1): 26,
    (1,2): 1e6,
    (2,2): 1e6,
    (3,2): 1e6,
    (4,2): 26,
    (1,3): 1e6, 
    (2,3): 26,
    (3,3): 1e6,
    (4,3): 26,
    (1,4): 1e6,   
    (2,4): 26,
    (3,4): 25,
    (4,4): 1e6,
    (1,5): 1e6,   
    (2,5): 17,
    (3,5): 18,
    (4,5): 1e6,
    (1,6): 1e6,
    (2,6): 1e6,
    (3,6): 18,
    (4,6): 21,
    (1,7): 1e6,
    (2,7): 17,
    (3,7): 18,
    (4,7): 18,
    (1,8): 1e6,
    (2,8): 1e6,
    (3,8): 18,
    (4,8): 1e6,
    (1,9): 26.5,
    (2,9): 1e6,
    (3,9): 1e6,
    (4,9): 1e6,
    (1,10): 26,
    (2,10): 1e6,
    (3,10): 1e6,
    (4,10): 1e6
}

# Capacidad máxima del almacén
Cap = 400   # ejemplo; ajusta a tu valor real

# Máximo unidades por entrega
MaxTrans = 8000

# Costo por entrega
C_trans = 4500

# Costo de plantación por unidad
C_plant = 20

# H(p): hectáreas de cada polígono 
H = {
    1: 5.4,
    2: 8.0,
    3: 8.0,
    4: 7.56,
    5: 7.11,
}

# HA(k): plantas de especie k por hectárea (Cuadro 2)
HA = {
    1: 33,
    2: 157,
    3: 33,
    4: 33,
    5: 39,
    6: 30,
    7: 58,
    8: 51,
    9: 69,
    10: 21
}

# Demanda d_{j,t,p}
# Por ejemplo: d[(j,t,p)] = demanda numérica
d = {}
for j in J:
    for t in T:
        for p in P:
            d[(j,t,p)] = H[p] * HA[j]  # <-- reemplaza 0 por la demanda concreta de j en t, polígono p

# Distancias entre polígonos (i, j) en el formato de un diccionario
dist = {}
for i in df_dist.index:
    for j in df_dist.columns:
        valor = df_dist.at[i, j]
        if pd.isna(valor):
            valor = 1e6  # o el valor que desees para distancias no definidas
        dist[(i, j)] = float(valor)

# Penalización por unidad no plantada
alpha = 1000  # pon aquí tu valor si existe (por unidad no plantada)

# Crear el problema PuLP
prob = pulp.LpProblem("Reforestacion_Logistica", pulp.LpMinimize)

### Variables de decisión

In [13]:
# (a) x_{i,j,t,p} >= 0: unidades de especie j que el proveedor i entrega en t al polígono p
x = {}
for i in I:
    for j in J:
        for t in T:
            for p in P:
                x[(i,j,t,p)] = pulp.LpVariable(
                    cat=pulp.LpContinuous,
                    name=f"x_{i}_{j}_{t}_{p}",
                    lowBound=0
                )

# (b) y_{j,t,p} >= 0: unidades de especie j plantadas en periodo t en polígono p
y = {}
for j in J:
    for t in T:
        for p in P:
            y[(j,t,p)] = pulp.LpVariable(
                cat=pulp.LpContinuous,
                name=f"y_{j}_{t}_{p}",
                lowBound=0
            )

# (c) z_{j,t} >= 0: inventario de especie j al final del periodo t
z = {}
for j in J:
    for t in T:
        z[(j,t)] = pulp.LpVariable(
            cat=pulp.LpContinuous,
            name=f"z_{j}_{t}",
            lowBound=0
        )

# (d) u_t ∈ Z+: número de entregas realizadas en periodo t
u = {}
for t in T:
    u[t] = pulp.LpVariable(
        cat=pulp.LpInteger,
        name=f"u_{t}",
        lowBound=0
    )

# (e) X_{l,j,p} >= 0: cantidad de plantas j llevadas al polígono p en viaje l
X = {}
for l in L:
    for j in J:
        for p in P:
            X[(l,j,p)] = pulp.LpVariable(
                cat=pulp.LpContinuous,
                name=f"X_{l}_{j}_{p}",
                lowBound=0
            )

# (f) Y_{i,j,l} ∈ {0,1}: 1 si se viaja del proveedor i al polígono j en el viaje l
Y = {}
for i in I:
    for j in P:
        for l in L:
            Y[(i,j,l)] = pulp.LpVariable(
                cat=pulp.LpBinary,
                name=f"Y_{i}_{j}_{l}"
            )

# (g) s_{j,t} >= 0: desviación positiva de demanda no satisfecha para especie j en t
s = {}
for j in J:
    for t in T:
        s[(j,t)] = pulp.LpVariable(
            cat=pulp.LpContinuous,
            name=f"s_{j}_{t}",
            lowBound=0
        )

# (h) Variable auxiliar w_{i,j,t,p} = x_{i,j,t,p} * Y_{i,p,1} (solo para l=1)
#      Usaremos l=1 en Y_{i,p,1} según el modelo.
w = {}
bigM = MaxTrans  # por ejemplo, Big‐M = capacidad máxima de entrega
for i in I:
    for j in J:
        for t in T:
            for p in P:
                w[(i,j,t,p)] = pulp.LpVariable(
                    cat=pulp.LpContinuous,
                    name=f"w_{i}_{j}_{t}_{p}",
                    lowBound=0
                )
DP = {}
for i in Nodos:
    for j in Nodos:
        for l in L:
            DP[(i,j,l)] = pulp.LpVariable(cat=pulp.LpBinary, name=f"DP_{i}_{j}_{l}")

### Función objetivo:

In [14]:
# Función Objetivo
#    min Z = sum_{i,j,t,p} C_{i,j} * x_{i,j,t,p}
#            + sum_t C_trans * u_t
#            + sum_{j,t,p} C_plant * y_{j,t,p}
#            + sum_{j,t} alpha * s_{j,t}
#            + sum_{i,j,t,p} w_{i,j,t,p}    (en reemplazo de x*Y para l=1)

# Término sum_{i,j,t,p} C_{i,j} * x_{i,j,t,p}
cost_sum_x = []
for (i,j,t,p), var_x in x.items():
    cij = C.get((i,j), 1e6)  # si (i,j) no existe, costo muy alto (no vende)
    cost_sum_x.append(cij * var_x)

# Término sum_t C_trans * u_t
cost_sum_u = [C_trans * u[t] for t in T]

# Término sum_{j,t,p} C_plant * y_{j,t,p}
cost_sum_y = [C_plant * y[(j,t,p)] for j in J for t in T for p in P]

# Término sum_{j,t} alpha * s_{j,t}
cost_sum_s = [alpha * s[(j,t)] for j in J for t in T]

# Término no lineal linealizado: sum_{i,j,t,p} w_{i,j,t,p}
cost_sum_w = [1.0 * w[(i,j,t,p)] for i in I for j in J for t in T for p in P]

cost_sum_dist = []
for l in L:
    for i in Nodos:
        for j in Nodos:
            costo_ij = dist.get((i,j), 1e6)
            cost_sum_dist.append(costo_ij * DP[(i,j,l)])

# Agregamos todos los trozos a la función objetivo:
prob += (
    pulp.lpSum(cost_sum_x)
    + pulp.lpSum(cost_sum_u)
    + pulp.lpSum(cost_sum_y)
    + pulp.lpSum(cost_sum_s)
    + pulp.lpSum(cost_sum_w)
    + pulp.lpSum(cost_sum_dist)
), "Costo_Total"

### Restricciones:

In [15]:
# Restricción (1): Balance de inventario
#     z_{j,t} = z_{j,t-1} + sum_i x_{i,j,t} - sum_p y_{j,t,p}, para todo j,t.
for j in J:
    for t in T:
        if t == 1:
            # Asumimos que z_{j,0} = 0 para todo j (inventario inicial cero).
            prob += (
                z[(j,t)]
                == 0
                + pulp.lpSum(x[(i,j,t,p)] for i in I for p in P)
                - pulp.lpSum(y[(j,t,p)] for p in P)
            ), f"BalanceInv_j{j}_t{t}"
        else:
            prob += (
                z[(j,t)]
                == z[(j,t-1)]
                + pulp.lpSum(x[(i,j,t,p)] for i in I for p in P)
                - pulp.lpSum(y[(j,t,p)] for p in P)
            ), f"BalanceInv_j{j}_t{t}"

# Restricción (2): Satisfacción de la demanda con desviación
#     sum_p y_{j,t,p} + s_{j,t} >= D_{j,t}   ∀ j,t
for j in J:
    for t in T:
        prob += (
            pulp.lpSum(y[(j,t,p)] for p in P) + s[(j,t)]
            >= sum(d[(j,t,p)] for p in P)
        ), f"SatisfaceDemanda_j{j}_t{t}"

# Restricción (3): Capacidad de almacenamiento
#     sum_j z_{j,t} <= Cap   ∀ t
for t in T:
    prob += (
        pulp.lpSum(z[(j,t)] for j in J) <= Cap
    ), f"CapAlmacen_t{t}"

# Restricción (4): Límite por entregas
#     sum_{i,j,p} x_{i,j,t,p} <= MaxTrans * u_t   ∀ t
for t in T:
    prob += (
        pulp.lpSum(x[(i,j,t,p)] for i in I for j in J for p in P)
        <= MaxTrans * u[t]
    ), f"LimiteEntrega_t{t}"

# Restricción (5): Tiempo mínimo de aclimatación
#     y_{j,t,p} <= z_{j, t-k}   ∀ j,t,p, para k = 3..7
# Nota: esta forma “∀ k=3..7” equivale a “y <= min(z_{j,t-3}, ..., z_{j,t-7})”.
#       Debemos imponerlo para cada k o, si queremos que cumpla cualquiera, usar 
#       y_{j,t,p} <= z_{j,t-3} AND ... AND y_{j,t,p} <= z_{j,t-7}. 
#       En PuLP se traducirá a 5 restricciones independientes:
for j in J:
    for t in T:
        for p in P:
            for k_aco in [3, 4, 5, 6, 7]:
                if t - k_aco >= 1:
                    prob += (
                        y[(j,t,p)] <= z[(j, t-k_aco)]
                    ), f"Aclimatacion_j{j}_t{t}_p{p}_k{k_aco}"
                else:
                    # Si t-k < 1, interpretamos que no se puede plantar antes de n días:
                    prob += (
                        y[(j,t,p)] == 0
                    ), f"NoPlantaAntesMin_j{j}_t{t}_p{p}_k{k_aco}"

# Restricción (6): No entregas en días de plantación
#     x_{i,j,t,p} = 0 si t es día de plantación.
#    Suponemos que existe un conjunto “DiasPlantacion ⊆ T” dado.
DiasPlantacion = {2, 4}  # por ejemplo; ajusta según tu caso real
for i in I:
    for j in J:
        for t in DiasPlantacion:
            for p in P:
                prob += (
                    x[(i,j,t,p)] == 0
                ), f"NoEntrega_DiaPlant_j{j}_i{i}_t{t}_p{p}"

# Restricción (7): Plantación según demanda por hectárea
#     sum_{l} X_{l,j,p} = floor( H(p) * HA(j) )   ∀ j,p
#    Redondeamos hacia abajo con int(...) en Python.
for j in J:
    for p in P:
        demanda_hectarea = int(H[p] * HA[j])
        prob += (
            pulp.lpSum(X[(l,j,p)] for l in L) == demanda_hectarea
        ), f"DemandaHectarea_j{j}_p{p}"

# Restricción (8): Capacidad del vehículo por viaje
#     sum_{j} X_{l,j,p} <= 524   ∀ l
for l in L:
    for p in P:
        prob += (
            pulp.lpSum(X[(l,j,p)] for j in J) <= 524
        ), f"CapVehiculo_l{l}_p{p}"

# Restricción (9): Relación entre viaje y cantidad transportada
#     sum_{j} X_{l,j,p} <= (sum_i Y_{i,p,l}) * 600   ∀ p,l
for p in P:
    for l in L:
        prob += (
            pulp.lpSum(X[(l,j,p)] for j in J)
            <= pulp.lpSum(Y[(i,p,l)] for i in I) * 600
        ), f"RelacionViajeCant_l{l}_p{p}"

# Restricción (10): Salida desde el almacén por viaje
#      sum_{j} Y_{a,j,l} = 1  ∀ l   (a = 1 es el almacén)
for l in L:
    prob += (
        pulp.lpSum(Y[(a,j,l)] for j in P) == 1
    ), f"SalidaAlmacen_l{l}"

# Restricción (11): Regreso al almacén por viaje
#      sum_{i} Y_{i,a,l} = 1  ∀ l
for l in L:
    prob += (
        pulp.lpSum(Y[(i,a,l)] for i in I) == 1
    ), f"RegresoAlmacen_l{l}"

# Restricción (12): Visitas válidas
#      sum_{j} Y_{i,j,l} <= sum_{i'} X_{i',l,i}  ∀ l, i>1
# Nota: interpretación: para visitar polígono i>1 en el viaje l, debe haber plantas asignadas 
#       a ese destino. Pero “sum_{i'} X_{i',l,i}” son plantas transportadas al polígono i, 
#       en todos los proveedores i'.  
for l in L:
    for i_poly in P:
        if i_poly != a:  # i_poly>1
            prob += (
                pulp.lpSum(Y[(i_poly,j,l)] for j in P)
                <= pulp.lpSum(X[(l,j,i_poly)] for j in J)
            ), f"VisitasValidas_l{l}_poly{i_poly}"

# Restricción (13): Una visita por polígono por viaje
#      sum_{j} Y_{i,j,l} <= 1   ∀ l, i
for l in L:
    for i_poly in P:
        prob += (
            pulp.lpSum(Y[(i_poly,j,l)] for j in P) <= 1
        ), f"UnaVisitaPorPol_l{l}_poly{i_poly}"

# Restricción (14): Control de disponibilidad con aclimatación
#      sum_{p∈P} y_{j,t,p} 
#        <= ( sum_{k=1 to t-n} sum_{i∈I} x_{j,i,k} )
#           − ( sum_{s=n+1 to t-1} sum_{p∈P} y_{j,s,p} ),  ∀ j ∈ J, t > n
n = 3  # ajusta n entre 3 y 7 según tu caso
for j in J:
    for t in T:
        if t > n:
            izquierda = pulp.lpSum(y[(j,t,p)] for p in P)
            parte1 = pulp.lpSum(x[(i,j,kp,p)] for i in I for kp in T if kp <= t-n for _ in [0] for _p in P)
            # Como x lleva índice (i,j,k,p): debemos sumar sobre p y i. Hacemos dos bucles:
            parte1 = pulp.lpSum(x[(i,j,kp,p)] for i in I for kp in T if kp <= t-n for p in P)
            parte2 = pulp.lpSum(y[(j,s,p)] for s in T if n+1 <= s <= t-1 for p in P)
            prob += (
                izquierda <= parte1 - parte2
            ), f"ControlAcli_j{j}_t{t}"
        else:
            # Para t <= n: sum_{p} y_{j,t,p} = 0
            prob += (
                pulp.lpSum(y[(j,t,p)] for p in P) == 0
            ), f"NoPlantaAntesMin_j{j}_t{t}"

# Restricción (16): n ∈ Z, 3 ≤ n ≤ 7  → ya hemos fijado n=3 (por ejemplo) arriba.

### Linealización

In [16]:
# Linealización del término w_{i,j,t,p} = x_{i,j,t,p} * Y_{i,p,1}
#    Para cada (i,j,t,p) imponemos:
#      w_{i,j,t,p} <= x_{i,j,t,p}
#      w_{i,j,t,p} <= M * Y_{i,p,1}
#      w_{i,j,t,p} >= x_{i,j,t,p} - M * (1 - Y_{i,p,1})
#    Con M = bigM = MaxTrans (8000)
for i in I:
    for j in J:
        for t in T:
            for p in P:
                # (a) w <= x
                prob += (
                    w[(i,j,t,p)] <= x[(i,j,t,p)]
                ), f"Lin1_w_{i}_{j}_{t}_{p}"
                # (b) w <= M * Y_{i,p,1}
                prob += (
                    w[(i,j,t,p)] <= bigM * Y[(i,p,1)]
                ), f"Lin2_w_{i}_{j}_{t}_{p}"
                # (c) w >= x - M * (1 - Y_{i,p,1})
                prob += (
                    w[(i,j,t,p)] >= x[(i,j,t,p)]
                    - bigM * (1 - Y[(i,p,1)])
                ), f"Lin3_w_{i}_{j}_{t}_{p}"

### Resolver:

In [17]:
prob.solve(pulp.PULP_CBC_CMD(msg=True))

# Imprimir el estado y los valores:

print("Status:", pulp.LpStatus[prob.status])
print("Valor óptimo de la función objetivo Z =", pulp.value(prob.objective))
print()

# Mostrar algunas variables no nulas (ejemplo):
print("Variables x_{i,j,t,p} distintas de cero:")
for (i,j,t,p), var in x.items():
    if var.varValue is not None and var.varValue > 1e-6:
        print(f"  x[{i},{j},{t},{p}] = {var.varValue}")

print("\nVariables y_{j,t,p} distintas de cero:")
for (j,t,p), var in y.items():
    if var.varValue is not None and var.varValue > 1e-6:
        print(f"  y[{j},{t},{p}] = {var.varValue}")

print("\nNúmero de entregas por período u_t:")
for t, var in u.items():
    if var.varValue is not None and var.varValue > 0:
        print(f"  u[{t}] = {var.varValue}")

# ... (puedes imprimir X, Y, s, w, z de forma similar si lo necesitas)

Status: Infeasible
Valor óptimo de la función objetivo Z = 136190385.0

Variables x_{i,j,t,p} distintas de cero:
  x[2,5,1,1] = 50.0
  x[2,5,5,4] = 150.0
  x[2,5,6,4] = 200.0

Variables y_{j,t,p} distintas de cero:
  y[5,8,1] = 50.0
  y[5,8,2] = 50.0
  y[5,8,3] = 50.0
  y[5,8,4] = 50.0
  y[5,9,1] = 50.0
  y[5,9,2] = 50.0
  y[5,9,3] = 50.0
  y[5,9,4] = 50.0

Número de entregas por período u_t:
  u[1] = 0.00625
  u[5] = 0.01875
  u[6] = 0.025
